Import Libs

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from preprocess_data import tfidf_vec_transform, tfidf_vec_fit_transform
from preprocess_data import count_vectorizer
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, model_multinominalNB_train, predict_values



Function for reading the data

In [ ]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [ ]:
## read data windows
data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
#data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.15, random_state=42, stratify=data_train_val['label'])


In [ ]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, ngram_range=(1,2))

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])

# calc tf-idf matrix with uncleand data
X_train_tfidf = vectorizer.fit_transform(data_train['text'])
X_val_tfidf = vectorizer.transform(data_val['text'])
X_test_tfidf = vectorizer.transform(data_test['text'])

#print("tfidf_matrix_train_val: \n", tfidf_matrix_train, "\n")
#print("tfidf_matrix_test: \n", tfidf_matrix_test, "\n")


# count vectorizer
#X_train_vec = count_vectorizer(data_train['lemmatized_text']).toarray()
#X_val_vec = count_vectorizer(data_val['lemmatized_text']).toarray()



In [ ]:
# Naive Bayes Model
model_nb = model_naives_bayes(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


In [ ]:
# Random Forest
model_rf = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

In [ ]:
# Multinomial Naive Bayes (MultinomialNB) classifier
model_mnb = model_multinominalNB_train(data_train['lemmatized_text'], data_val['lemmatized_text'], data_train['label'], data_val['label'])


In [ ]:
# Simple Feedforward NN -> no good results
model_sfnn_train(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


In [ ]:
model_lr = model_logistic_regression(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

In [ ]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

#predict_values(model_rf, tfidf_matrix_test, data_test, filepath)
predict_values(model_nb, tfidf_matrix_test, data_test, filepath)